<a href="https://colab.research.google.com/github/lucasquemelli/House_Rocket_Machine_Learning/blob/main/House_Rocket_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px

from scipy import stats
from matplotlib import pyplot as plt